In [ ]:
from datascience import *

In [ ]:
import cv2
import matplotlib as plt
import os
import pandas as pd

In [ ]:
g = '707_01-17-2016_2976_Memphis Grizzlies_3173_New York Knicks_period1'
game_date = g.split('_')[1].replace('-', '.')
vis = g.split('_')[3]
home = g.split('_')[5]
game_date, vis, home

In [ ]:
team_acr = """
ATL\tAtlanta Hawks
BOS	Boston Celtics
BKN	Brooklyn Nets
CHA	Charlotte Hornets
CHI	Chicago Bulls
CLE	Cleveland Cavaliers
DAL	Dallas Mavericks
DEN	Denver Nuggets
DET	Detroit Pistons
GSW	Golden State Warriors
HOU	Houston Rockets
IND	Indiana Pacers
LAC	Los Angeles Clippers
LAL	Los Angeles Lakers
MEM	Memphis Grizzlies
MIA	Miami Heat
MIL	Milwaukee Bucks
MIN	Minnesota Timberwolves
NOP	New Orleans Pelicans
NYK	New York Knicks
OKC	Oklahoma City Thunder
ORL	Orlando Magic
PHI	Philadelphia 76ers
PHX	Phoenix Suns
POR	Portland Trail Blazers
SAC	Sacramento Kings
SAS	San Antonio Spurs
TOR	Toronto Raptors
UTA	Utah Jazz
WAS	Washington Wizards
"""

acr_dict = {}
team_acr = team_acr.split('\n')[1: -1]
for row in team_acr:
    # print(row.split('\t'))
    full = row.split('\t')[1]
    acr = row.split('\t')[0]
    acr_dict[full] = acr
acr_dict

In [ ]:
import os

fp = r"G:\Other computers\mac_new\NBA_HUDL_data\15-16\720"

def get_vids_path_map(fp):
    """
    Given a directory path to videos from the 15-16 NBA season, 
    returns a dict mapping formatted game titles to abs video paths.
    """

    game_vids = os.listdir(fp)
    game_vids_map = {}
    for game in game_vids:
        gp = os.path.join(fp, game)
        game_date = game.split('_')[1].replace('-', '.')
        home = game.split('_')[3]
        vis = game.split('_')[5]
        game_date, vis, home
        formatted_title = f'{game_date}.{acr_dict[vis]}.at.{acr_dict[home]}.7z'
        game_vids_map[formatted_title] = gp
    return game_vids_map

vids_paths_map = get_vids_path_map(fp)


In [ ]:
from datetime import datetime, timedelta

games = set(os.listdir("2016.NBA.Raw.SportVU.Game.Logs"))
# print(games)
# print(map)

# game_titles: video_paths

def fix_game_date_offsets(statvu_logs_dir, videos_dir, videos_paths_map: dict[str: str]):

    games = set(os.listdir(statvu_logs_dir))
    for k in videos_paths_map.keys():

        date_str = k[0:10]
        date = k[0:10]
        date = datetime.strptime(date, '%m.%d.%Y')

        # calculate +- one day
        date_plus = date + timedelta(days=1)
        date_minus = date - timedelta(days=1)
        date_plus = date_plus.strftime('%m.%d.%Y')
        date_minus = date_minus.strftime('%m.%d.%Y')

        if k in games:
            pass
        elif k.replace(date_str, date_minus) in games:
            path = videos_paths_map.pop(k)
            videos_paths_map[k.replace(date_str, date_minus)] = path
        elif k.replace(date_str, date_plus) in games:
            path = videos_paths_map.pop(k)
            videos_paths_map[k.replace(date_str, date_plus)] = path


fix_game_date_offsets('2016.NBA.Raw.SportVU.Game.Logs', fp, vids_paths_map)


In [ ]:
# [game_log_path, video_game_path]
# 

games_videos_paths = []
for k in vids_paths_map:
    statvu_data_path = os.path.join("2016.NBA.Raw.SportVU.Game.Logs", k)
    video_path = vids_paths_map[k]

    assert os.path.isfile(video_path)
    if os.path.isfile(statvu_data_path):
        games_videos_paths.append([statvu_data_path, video_path])


In [ ]:
import os
import py7zr

dir_path = '2016.NBA.Raw.SportVU.Game.Logs'
games = os.listdir(dir_path)

for game in games:
    gp = os.path.join(dir_path, game)

    # Ensure the file is a .7z file before trying to unzip
    if gp.endswith('.7z'):
        with py7zr.SevenZipFile(gp, mode='r') as archive:
            archive.extractall(path=dir_path)

        # Rename extracted JSON files
        extracted_files = os.listdir(dir_path)
        for f in extracted_files:
            if f.endswith('.json') and '.201' not in f:
                os.rename(
                    os.path.join(dir_path, f), 
                    os.path.join(
                        dir_path, game.replace('.7z', '.json')
                    )
                )
                break


In [ ]:
# 1. extract timestamps from game
# 2. match frames to moments
# 3. save new data and upload both versions (w/ frames, w/o frames)
# 4. produce visualized results

In [ ]:
# TODO: 
 # 1. fix text extraction errors
 # 2. Improve visualization quality + speed of rendering
 # 3. Produce many visualizations and moments files + save and upload

# !pip install -q ultralytics
# !pip install -q paddlepaddle
# !pip install -q paddleocr
# !tail -f process.log

from temporal_grounding import *

vp = r"demo\17769_12-27-2015_3175_Phoenix Suns_3224_Philadelphia 76ers_period2.mp4"
dp = r"demo\01.01.2016.NYK.at.CHI.json"
extract_timestamps_from_video(vp, 'demo/12.26.2015.PHI.at.PHX.timestamps.json')

In [ ]:
import json

fp = r'demo/01.01.2016.NYK.at.CHI.json'
with open(fp, 'r') as f:
    data = json.load(f)

In [ ]:
import json

def get_unique_moments_from_statvu(statvu_log_path):
    """
    Extracts unique moments from a StatVu log file.

    Args:
    statvu_log_path (str): The file path of the StatVu log.

    Returns:
    dict: A dictionary mapping unique quarter-time remaining combinations to their respective moment details.
    """
    with open(statvu_log_path, 'r') as file:
        data = json.load(file)

    unique_quarter_time_combinations = set()
    processed_moments = {}

    for event in data['events']:
        for moment in event['moments']:
            quarter, moment_id, time_remaining_quarter, time_remaining_shot_clock, _, positions = moment
            moment_identifier = f"{quarter}_{time_remaining_quarter}"
            if moment_identifier not in unique_quarter_time_combinations:
                player_positions = [
                    {'team_id': player_data[0], 'player_id': player_data[1], 
                     'x_position': player_data[2], 'y_position': player_data[3], 'z_position': player_data[4]}
                    for player_data in positions
                ]
                processed_moments[moment_identifier] = {
                    'quarter': quarter,
                    'moment_id': moment_id,
                    'time_remaining_in_quarter': time_remaining_quarter,
                    'time_remaining_on_shot_clock': time_remaining_shot_clock,
                    'player_positions': player_positions
                }
                unique_quarter_time_combinations.add(moment_identifier)

    return processed_moments

fp = r"demo\12.26.2015.PHI.at.PHX.json"
m = get_unique_moments_from_statvu(fp)

In [ ]:
import math

fp = r'demo/timestamps.json'
with open(fp, 'r') as f:
    data = json.load(f)

# statvu format
    # 1/25 fps
    # -1/25s per frame

# 1. interpolate between time_remaining values

tr_vals = []
for k in data:
    tr = data[k]['time_remaining']
    if tr == None:
        tr_vals.append(-1)
    else:
        tr_vals.append(tr)

last_seen = -1
last_seen_cnt = 0
inter_vals = []
for v in tr_vals:
    if v > 0:
        if last_seen_cnt == 0 or last_seen_cnt > 30:
            last_seen = v
            last_seen_cnt = 0
            inter_vals.append(v)
            last_seen_cnt += 1
        else:
            # 26 / 30 = x / 25
            x = math.floor(((last_seen_cnt) / 30) * 25) # multiplier
            v_temp = v - (x / 25)
            v_temp = round(v_temp, 2)
            last_seen_cnt += 1
            inter_vals.append(v_temp)
    else:
        inter_vals.append(None)

for k, v in zip(data, inter_vals):
    data[k]['time_remaining'] = v

with open(r"demo\timestamps_mod.json", "w") as outfile: 
    json.dump(data, outfile, indent=4)

In [ ]:
import json
import math

def interpolate_timestamps(file_path, output_path):
    """
    Interpolates time remaining values in a JSON file containing timestamp data.

    Args:
    file_path (str): The file path of the input JSON file with timestamp data.
    output_path (str): The file path for the output JSON file with interpolated timestamps.

    The function reads the timestamp data, interpolates the 'time_remaining' values, and writes
    the updated data to a new JSON file.
    """

    # Load data from the file
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return

    # Initialize list to store original time_remaining values
    original_time_values = []

    # Extract time_remaining values and handle None values
    for key in data:
        time_remaining = data[key].get('time_remaining', -1)
        original_time_values.append(time_remaining if time_remaining is not None else -1)

    # Initialize variables for interpolation
    last_seen_valid_time = -1
    consecutive_frame_count = 0
    interpolated_values = []

    # Perform interpolation of time_remaining values
    for value in original_time_values:
        if value > 0:
            if consecutive_frame_count == 0 or consecutive_frame_count > 30:
                last_seen_valid_time = value
                consecutive_frame_count = 0
            else:
                multiplier = math.floor((consecutive_frame_count / 30) * 25)
                interpolated_value = round(value - (multiplier / 25), 2)
                interpolated_values.append(interpolated_value)
            consecutive_frame_count += 1
        else:
            interpolated_values.append(None)

    # Update the data dictionary with interpolated values
    for key, interpolated_value in zip(data, interpolated_values):
        data[key]['time_remaining'] = interpolated_value

    # Write the updated data to a new JSON file
    try:
        with open(output_path, "w") as outfile:
            json.dump(data, outfile, indent=4)
    except Exception as e:
        print(f"Error writing file {output_path}: {e}")

# Example usage:
interpolate_timestamps(r'demo\12.26.2015.PHI.at.PHX.timestamps.json', r'demo\12.26.2015.PHI.at.PHX.timestamps.mod.json')

In [61]:
from datetime import timedelta

# def interpolate_timestamps(src_fp, dst_fp):
    
src_fp = r"demo\12.26.2015.PHI.at.PHX.timestamps.json"
with open(src_fp, 'r') as f:
    timestamps = json.load(f)

tr = []
tr_mod = []

for k in timestamps:
    tr.append(timestamps[k]['time_remaining'])

is_dec = False
dec_index = 0

seen = set()
for i in range(0, len(tr) - 30):

    curr = tr[i]
    next = tr[i + 30] # look ahead 30 frames
    # print(curr)
    # i / 30 = x / 25
    # floor((i / 30)) * 25 = x
    # dec = x / 25


    temp = 0

    if curr != None:
        if curr not in seen and next != None and next < curr:
            is_dec = True
            dec_index = 1
            seen.add(curr)
            tr_mod.append(curr)
        elif is_dec:
            tr_mod.append(round(curr - ((1/30) * dec_index), 2))
            dec_index += 1
        else:
            dec_index = 1
            tr_mod.append(curr)
    else:
        is_dec = False
        dec_index = 1
        tr_mod.append(curr)

for k, v in zip(timestamps, tr_mod):
    timestamps[k]['time_remaining'] = v

with open(r'demo\12.26.2015.PHI.at.PHX.timestamps.mod.json', 'w') as f:
    json.dump(timestamps, f, indent=4)

    

In [62]:
import tqdm

# 1. extract timestamps from video
# 2. interpolate and save timestamps
# 3. map video frames to moments

def map_frames_to_moments(data, moments_data):
    """
    Maps frames in 'data' to their corresponding moments in 'moments_data' based on time proximity.

    Args:
    data (dict): A dictionary containing data with keys indicating frame identifiers and values 
                 having 'quarter' and 'time_remaining' information.
    moments_data (dict): A dictionary where keys are string representations of 'quarter_time' and values
                         are the moments to map.

    Returns:
    dict: A dictionary mapping frame identifiers to corresponding moments in 'moments_data'.
    """

    def is_close(time1, time2, tolerance=0.2 ):
        """Check if two time values are within a given tolerance."""
        return abs(time1 - time2) <= tolerance

    frames_matched = 0
    total_frames = 0
    frames_moments_map = {}

    moments_dict = {}
    for moment_key in moments_data:
        quarter, time_remaining = map(float, moment_key.split('_'))
        if quarter not in moments_dict:
            moments_dict[quarter] = []
        moments_dict[quarter].append(time_remaining)

    for frame_id in tqdm.tqdm(data):
        quarter_time_key = str(data[frame_id]['quarter']) + '_' + str(data[frame_id]['time_remaining']) if data[frame_id]['time_remaining'] != None else None
        if quarter_time_key:
            total_frames += 1
            quarter, time_remaining = map(float, quarter_time_key.split('_'))

            match_found = False
            if quarter in moments_dict:
                closest_time = None
                min_difference = float('inf')
                for moment_time in moments_dict[quarter]:
                    difference = abs(time_remaining - moment_time)
                    if difference < min_difference:
                        min_difference = difference
                        closest_time = moment_time

                if is_close(time_remaining, closest_time):
                    frames_matched += 1
                    match_found = True
                    moment_key = f"{int(quarter)}_{closest_time}"
                    frames_moments_map[frame_id] = moments_data[moment_key]
            if not match_found:
                frames_moments_map[frame_id] = None
        else:
            frames_moments_map[frame_id] = None

    print(frames_matched, '/', total_frames)
    return frames_moments_map

# Example usage:
with open(r'demo\12.26.2015.PHI.at.PHX.timestamps.mod.json', 'r') as f:
    data = json.load(f)
frames_moments_map = map_frames_to_moments(data, m)

100%|██████████| 35657/35657 [00:40<00:00, 886.04it/s]

32078 / 34862


In [63]:
with open('demo/moments.json', 'w') as f:
    json.dump(frames_moments_map, f, indent=4)

In [ ]:
# TODO:
    # 1. Generate a viz for demo game
    # 2. Write a summary report + upload changes
    # 3. Apply for housing
    # 4. Work on takehome exam
    # 5. Read ViT review
    # 6. Evening meditation

In [71]:
from matplotlib import pyplot as plt
import cv2
import json
import numpy as np
from tqdm import tqdm

# Load JSON data
moments_path = 'demo/moments.json'  # Update this to the correct path to your moments.json file
with open(moments_path, 'r') as file:
    player_data = json.load(file)

# Load the basketball court image
court_path = 'court.png'  # Update this to the correct path to your court.png file
court_img = plt.imread(court_path)

# Dimensions of the court on the image
court_width, court_height = 939, 500

# Initialize video capture
video_path = r'demo\17769_12-27-2015_3175_Phoenix Suns_3224_Philadelphia 76ers_period2.mp4'  # Update this to the correct path to your video file
cap = cv2.VideoCapture(video_path)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Initialize video writer
output_path = r'demo\viz.mp4'  # Update this to your desired output path
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, frame_rate, (2*frame_width, frame_height))

def format_time(seconds):
    """Convert time in seconds to minutes:seconds format."""
    minutes = int(seconds) // 60
    seconds = seconds % 60
    return f"{minutes:02d}:{seconds:05.2f}"

# Function to draw player positions on the court
def plot_positions_on_court(moment_data, ax):
    if moment_data:
        for player in moment_data['player_positions']:
            x = (player['x_position'] / 100) * court_width
            y = (player['y_position'] / 50) * court_height
            color = 'red' if player['team_id'] == 1610612756 else 'blue' if player['team_id'] == 1610612755 else 'white'
            ax.plot(x, y, 'o', color=color, markersize=15)

        # Display the time remaining in the quarter
        time_remaining = moment_data['time_remaining_in_quarter']
        time_text = format_time(time_remaining)
        ax.text(0.5, 0.02, f"Time Remaining: {time_text}", transform=ax.transAxes, 
                horizontalalignment='center', fontsize=12, bbox=dict(facecolor='black', alpha=0.5))

start_time = 2 * 60 + 10  # 10 minutes and 10 seconds
end_time = 2 * 60 + 30    # 10 minutes and 20 seconds
start_frame = start_time * frame_rate
end_frame = end_time * frame_rate

for frame_index in tqdm(range(0, end_frame), desc="Processing Frames"):
    ret, frame = cap.read()
    if not ret:
        break

    # Skip frames outside the desired range
    if frame_index < start_frame or frame_index > end_frame:
        continue

    fig, ax = plt.subplots(figsize=(court_width/100, court_height/100), dpi=100)
    ax.imshow(court_img)
    ax.axis('off')  # Turn off axis
    
    moment_data = player_data.get(str(frame_index))
    plot_positions_on_court(moment_data, ax)

    fig.canvas.draw()
    plot_image = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    plot_image = plot_image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    plot_image_resized = cv2.resize(plot_image, (frame_width, frame_height))

    combined_frame = np.hstack((frame, plot_image_resized))
    out.write(combined_frame)

    plt.close(fig)

cap.release()
out.release()  # Release the video writer

Processing Frames: 100%|██████████| 4500/4500 [00:58<00:00, 76.41it/s] 


In [ ]:
from viz import *

In [42]:
visualize_timestamps(
    r"demo\17769_12-27-2015_3175_Phoenix Suns_3224_Philadelphia 76ers_period2.mp4", 
    r"demo\12.26.2015.PHI.at.PHX.timestamps.json",
    r"viz.mp4",
)

100%|██████████| 35657/35657 [06:52<00:00, 86.47it/s] 
